In [63]:
import os
import pandas as pd
import requests
from bs4 import BeautifulSoup

In [64]:
# SETTING URLS 
program_urls = {'us':"https://ideas.repec.org/top/top.usecondept.html",
                'urban':"https://ideas.repec.org/top/top.ure.html",
                'macro':"https://ideas.repec.org/top/top.mac.html",
                'public':"https://ideas.repec.org/top/top.pub.html"}

# COLLECTING RAW HTML TEXT FROM PAGE 
program_html = {}
for emphasis,url in program_urls.items():
    program_html[emphasis] = requests.get(url).text

# PARSING HTML
program_soup = {}
for emphasis, html in program_html.items():
    program_soup[emphasis] = BeautifulSoup(html, "html.parser")

In [65]:
# ORGANIZING RAW HTML INTO EMPHASIS SCHOOL RANKINGS
rankings_raw = {}

for emphasis, soup in program_soup.items():
    # For particular emphasis we find the number of rows in the table
    n_rows = len(soup.find_all('table')[1].find_all('tr'))
    rank = []
    schools = []
    
    # For number of rows, we find the school name and assign it a rank
    for row in range(1,n_rows):
        schools.append(soup.find_all('table')[1].find_all('tr')[row].find_all('td')[1].text)
        rank.append(row)
    
    # Construct nested dict for each emphasis made up of the school and its rank    
    rankings_raw[emphasis] = {'schools':schools, 'rank':rank}

In [66]:
# CLEANING INTO US ONLY DATAFRAMES
dataframes_raw = {}

for emphasis, info in rankings_raw.items():
    temp_df = pd.DataFrame(info)
    temp_df = temp_df[temp_df['schools'].apply(lambda x: "(USA)" in x)]
    temp_df = temp_df.reset_index()
    temp_df[f'us_{emphasis}_rank'] = temp_df.index + 1
    temp_df = temp_df.drop(columns=['index'])
    
    dataframes_raw[emphasis] = temp_df

In [67]:
# EXPORTING RAW INTO CSVs
for emphasis, df in dataframes_raw.items():
    df.to_csv(f'../data/raw/{emphasis}_rankings_raw.csv')
    
# CONSTRUCTING OVERACHIEVERS COLUMN and EXPORTING INTO CSVs
us_raw = dataframes_raw.pop('us')
us_raw = us_raw.rename(columns={'us_us_rank':'us_overall_rank'})
for emphasis, df in dataframes_raw.items():
    merged = us_raw.merge(df, how='left', on='schools')
    merged['rank_diff'] = merged['us_overall_rank'] - merged[f'us_{emphasis}_rank']
    merged = merged.sort_values('rank_diff', ascending=False)
    merged.set_index('schools', inplace=True)
    merged.to_csv(f'../data/processed/{emphasis}_overperformers.csv')